<a href="https://colab.research.google.com/github/Fuenfgeld/HospitalNavigator/blob/main/HospitalNavPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---

### To initialize the code click Runtime and then Run all
<img src="https://github.com/Fuenfgeld/HospitalNavigator/raw/main/img/RunAll.png" width="520" height="132" alt="Run All"/>


---


### Every Cell can be run individualy by clicking the Play Button
<img src="https://github.com/Fuenfgeld/HospitalNavigator/raw/main/img/RunCell.png" width="295" height="88" />


---



In [1]:
import folium
from folium import plugins
import pandas as pd

import ipywidgets
import os
import json

from IPython.display import YouTubeVideo
import datetime

## To learn the basics about geo visualisation in Python check out this Video about the cholera outbreak in broad street

In [2]:
!git clone https://github.com/Fuenfgeld/HospitalNavigator.git

Cloning into 'HospitalNavigator'...



---
## Intitialize Map
---

In [3]:
# the latitude and Longitude coordinates
UMMlocation = (17.54160754716436, 78.38678238173406)
map_UMM = folium.Map(location = UMMlocation, width = "100%", zoom_start = 17) # max zoom: 18
map_UMM

---
## Display geojson Outline
---
geojson drawing tool: http://geojson.io/

In [4]:
hauseOutline = 'GeoResources/D-block.geojson'
display(folium.GeoJson(hauseOutline, name="D-block").add_to(map_UMM))
display(map_UMM)

---
##Display Ant Path
---

In [5]:
testGeoJson = 'GeoResources/path/dpath.geojson'

In [6]:
def switchPosition(coordinate):
  temp = coordinate[0]
  coordinate[0] = coordinate[1]
  coordinate[1] = temp
  return coordinate

In [7]:
with open(testGeoJson) as f:
  testWay = json.load(f)

for feature in testWay['features']:
    path = feature['geometry']['coordinates']
finalPath = list(map(switchPosition,path))
finalPath

[[17.53981400625817, 78.38630281147084],
 [17.539154183818596, 78.38566617738934],
 [17.53874949153476, 78.38521407492641],
 [17.537931306548273, 78.38514948886035],
 [17.53725388177591, 78.38488191801497],
 [17.536892344630303, 78.38469746196455],
 [17.536618754111785, 78.38492290640812]]

In [8]:
path = 'GeoResources/path/dpath.geojson'
folium.plugins.AntPath([[17.53981400625817, 78.38630281147084],
 [17.539154183818596, 78.38566617738934],
 [17.53874949153476, 78.38521407492641],
 [17.537931306548273, 78.38514948886035],
 [17.53725388177591, 78.38488191801497],
 [17.536892344630303, 78.38469746196455],
 [17.536618754111785, 78.38492290640812]]).add_to(map_UMM)
map_UMM



---
## Build Navigator Prototype
---




In [9]:
class navigator:
    def __init__(self):
        self.geoResources = {}
        self.hospitalLocation =(49.49226339787344, 8.487303256988524)
        self.position = 'w'
        self.destination = 'A-path'

        for root, dirs, files in os.walk('HospitalNavigator/GeoResources'):  
            for file in files:
                self.geoResources[file.split('.')[0]] = root+'/'+file

    def changeDestination(self,newDestination):
        self.destination = newDestination
        self.redrawMap()

    def changeStartPoint(self, newStartPoint):
        
        #self.position = newStartPoint #does not work yet
        print(f'Selected Start: {newStartPoint}; Selected Target: {self.destination}')
        #self.redrawMap()
        

    def drawPathWay(self,hospitalMap):
      
      def switchPosition(coordinate):
        temp = coordinate[0]
        coordinate[0] = coordinate[1]
        coordinate[1] = temp
        return coordinate

      searchString = self.position + self.destination.split('Haus')[1]
      with open(self.geoResources[searchString]) as f:
           testWay = json.load(f)

      for feature in testWay['features']:
        path = feature['geometry']['coordinates']

      finalPath = list(map(switchPosition,path))
      folium.plugins.AntPath(finalPath).add_to(hospitalMap)

    def drawBuilding(self,hospitalMap):
      hauseOutline = self.geoResources[self.destination]
      folium.GeoJson(hauseOutline, name="geojson").add_to(hospitalMap)

    def redrawMap(self):
        #print(f'position {self.position}, destination {self.destination}')
        hospitalMap = folium.Map(location = self.hospitalLocation, width = "75%", zoom_start = 17)
        self.drawPathWay(hospitalMap)
        self.drawBuilding(hospitalMap)
        display(hospitalMap)




In [10]:
myNavigator = navigator()

In [11]:
def displayWay(whereTo):
     myNavigator.changeDestination(whereTo)

In [12]:
def changePosition(whereFrom):
    myNavigator.changeStartPoint(whereFrom)

In [17]:
import ipywidgets
from IPython.display import display
import folium
import folium.plugins
import json

# The central location
UMMlocation = (17.54160754716436, 78.38678238173406)

# Dictionary to hold the GeoJSON file paths for different locations
geojson_paths = {
     ' ': '',
    'A-block ': 'GeoResources/path/A-path.geojson',
    'B-block ': 'GeoResources/path/B-blockway.geojson',
    'C-block': 'GeoResources/path/B-101path.geojson',
    'D-block ': 'GeoResources/path/dpath.geojson',
    'E-block ': 'GeoResources/path/E-block_path.geojson',
    'ground1 ': 'GeoResources/path/ground1-path.geojson',
    'ground2 ': 'GeoResources/path/ground2-path.geojson',
    'ks audi': 'GeoResources/path/kspath.geojson',
    'peb-block': 'GeoResources/path/pebpath.geojson',
    'peb-block2': 'GeoResources/path/PG-PATH.geojson',

    # Add more paths as needed
}

def switchPosition(coordinate):
    """Swaps the latitude and longitude in a coordinate pair."""
    return [coordinate[1], coordinate[0]]

def load_geojson_coordinates(geojson_file):
    """Loads and processes coordinates from a GeoJSON file, ensuring LineString."""
    try:
        with open(geojson_file) as f:
            data = json.load(f)
        all_coordinates = []
        for feature in data['features']:
            if feature['geometry']['type'] == 'LineString':
                path = feature['geometry']['coordinates']
                final_path = list(map(switchPosition, path))
                all_coordinates.extend(final_path)
            elif feature['geometry']['type'] == 'Polygon':
                # Extract the outer boundary coordinates for AntPath
                path = feature['geometry']['coordinates'][0]
                final_path = list(map(switchPosition, path))
                all_coordinates.extend(final_path)
        return all_coordinates
    except FileNotFoundError:
        print(f"Error: File not found at {geojson_file}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {geojson_file}")
        return None

# Global variable to hold the current map display
map_display_handle = None

# Function to update and display the map based on the selected value
def update_map(selected_location):
    global map_display_handle

    new_map = folium.Map(location=UMMlocation, width="100%", zoom_start=17)

    if selected_location in geojson_paths:
        filepath = geojson_paths[selected_location]
        path_coordinates = load_geojson_coordinates(filepath)
        if path_coordinates:
            folium.plugins.AntPath(path_coordinates, name=selected_location).add_to(new_map)
    else:
        print(f"No GeoJSON path defined for {selected_location}")

    # Display the new map, replacing the old one if it exists
    if map_display_handle:
        map_display_handle.update(new_map)
    else:
        map_display_handle = display(new_map, display_id=True)

# Create the widget options
widget_options = list(geojson_paths.keys())

# Create the Select widget
location_widget = ipywidgets.Select(
    options=widget_options,
    value=widget_options[0] if widget_options else None,
    description='Select Location:',
    disabled=False
    
  
    
)

# Interacting with the widget
def on_location_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_map(change['new'])

location_widget.observe(on_location_change, names='value')

# Initial display
update_map(location_widget.value)
display(location_widget)

Error: File not found at 


Select(description='Select Location:', options=(' ', 'A-block ', 'B-block ', 'C-block', 'D-block ', 'E-block '…

In [ ]:
import ipywidgets
from IPython.display import display
import folium
import folium.plugins
import json

# The central location
UMMlocation = (17.54160754716436, 78.38678238173406)

# Dictionary to hold the GeoJSON file paths for different locations
geojson_paths = {
    ' ': '',
    'A-block ': 'GeoResources/path/A-path.geojson',
    'B-block ': 'GeoResources/path/B-blockway.geojson',
    'C-BLOCK': 'GeoResources/path/B-101path.geojson',
    'D-block ': 'GeoResources/path/dpath.geojson',
    'E-BLOCK': 'GeoResources/path/E-block_path.geojson',
    'ground1 ': 'GeoResources/path/ground1-path.geojson',
    'ground2 ': 'GeoResources/path/ground2-path.geojson',
    'ks audi': 'GeoResources/path/kspath.geojson',
    'peb-block': 'GeoResources/path/pebpath.geojson',
    'CANTEEN':'GeoResources/path/catten_path.geojson',
    'PG-BLOCK': 'GeoResources/path/PG-PATH.geojson',

    # Add more paths as needed
}

def switchPosition(coordinate):
    """Swaps the latitude and longitude in a coordinate pair."""
    return [coordinate[1], coordinate[0]]

def load_geojson_coordinates(geojson_file):
    """Loads and processes coordinates from a GeoJSON file, ensuring LineString."""
    try:
        with open(geojson_file) as f:
            data = json.load(f)
        all_coordinates = []
        for feature in data['features']:
            if feature['geometry']['type'] == 'LineString':
                path = feature['geometry']['coordinates']
                final_path = list(map(switchPosition, path))
                all_coordinates.extend(final_path)
            elif feature['geometry']['type'] == 'Polygon':
                # Extract the outer boundary coordinates for AntPath
                path = feature['geometry']['coordinates'][0]
                final_path = list(map(switchPosition, path))
                all_coordinates.extend(final_path)
        return all_coordinates
    except FileNotFoundError:
        print(f"Error: File not found at {geojson_file}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {geojson_file}")
        return None

# Global variable to hold the current map display
map_display_handle = None

# Function to update and display the map based on the selected value
def update_map(selected_location):
    global map_display_handle

    new_map = folium.Map(location=UMMlocation, width="100%", zoom_start=17)

    if selected_location in geojson_paths:
        filepath = geojson_paths[selected_location]
        path_coordinates = load_geojson_coordinates(filepath)
        if path_coordinates:
            folium.plugins.AntPath(path_coordinates, name=selected_location).add_to(new_map)
    else:
        print(f"No GeoJSON path defined for {selected_location}")

    # Display the new map, replacing the old one if it exists
    if map_display_handle:
        map_display_handle.update(new_map)
    else:
        map_display_handle = display(new_map, display_id=True)

# Create the widget options
widget_options = list(geojson_paths.keys())

# Create the Select widget
location_widget = ipywidgets.Select(
    options=widget_options,
    value=widget_options[0] if widget_options else None,
    disabled=False,
    layout=ipywidgets.Layout(
        width='200px'  # Set the width of the dropdown
    )
)

# Create a label for the dropdown with larger font and color
label_widget = ipywidgets.HTML(
    value="<b style='font-size:20px; color:blue;'>Select Location:</b>"
)

# Combine the label and dropdown in a vertical layout
container = ipywidgets.VBox(
    [label_widget, location_widget],
    layout=ipywidgets.Layout(
        align_items='flex-start'  # Align the label and dropdown to the left
    )
)

# Interacting with the widget
def on_location_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_map(change['new'])

location_widget.observe(on_location_change, names='value')

# Initial display
update_map(location_widget.value)
display(container)

Error: File not found at 
